# Practice using F1 data from Ergast API (compiled by Kaggle)
[F1 World Championship 1950-2023](https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020)

In [2]:
# Load packages
import pandas as pd
import zipfile
import os
from sqlalchemy import create_engine

# Connect to an in-memory SQLite database
engine = create_engine('sqlite:///:memory:')

# Configure the SQL magic
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Use %sql magic to connect to the SQLite database
%sql sqlite:///:memory:

# Unzip the data
with zipfile.ZipFile('F1.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

# Load each CSV into a DataFrame
tables = []
for f in os.listdir('data'):
    if f.endswith('.csv'):
        table_name = f.replace('.csv', '')
        file_path = os.path.join('data', f)
        # tables[table_name] = pd.read_csv(file_path)
        tables.append(table_name)
        globals()[table_name] = pd.read_csv(file_path)


# Use %sql magic to create temporary tables
for table_name in tables:
    %sql DROP TABLE IF EXISTS {table_name}
    %sql --persist {table_name}

In [4]:
# Query using %sql magic
res = %sql SELECT * FROM results LIMIT 5
display(res)


,index,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


In [3]:
%sql SELECT * FROM circuits LIMIT 5

,index,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park


In [9]:
%%sql df << 
SELECT * FROM drivers LIMIT 5

Returning data to local variable df


In [10]:
%%sql
select * 
from drivers 
join results using(driverId)
where driverRef = "hamilton"

,index,driverId,driverRef,number,code,forename,surname,dob,nationality,url,...,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,...,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,...,5,4.0,56,+46.548,5525103,53,3,1:35.462,209.033,1
2,0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,...,13,0.0,56,\N,\N,25,19,1:35.520,203.969,11
3,0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,...,3,6.0,66,+4.187,5903238,20,3,1:22.017,204.323,1
4,0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,...,2,8.0,58,+3.779,5213230,31,2,1:26.529,222.085,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,...,3,15.0,70,+4.598,5642946,47,2,1:15.396,208.228,1
318,0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,...,8,4.0,71,+49.196,5182803,49,10,1:09.283,224.366,1
319,0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,...,3,15.0,52,+6.783,5123721,43,3,1:30.545,234.221,1
320,0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,...,4,12.0,70,+39.134,5927768,54,2,1:21.601,193.277,1
